In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# 生成器GのクラスGenerator
class Generator(nn.Module):
    """
    生成器Gのクラス
    """
    def __init__(self, nz=100, nch_g=128, nch=1):
        """
        Args:
            nz (int, optional): 入力ベクトルzの次元数. Defaults to 100.
            nch_g (int, optional): 最終層の入力チャンネル数. Defaults to 128.
            nch (int, optional): 出力画像のチャンネル数. Defaults to 1.
        """
        super().__init__()

        # ニューラルネットワークの構造定義
        self.layers = nn.ModuleDict({
            'layer0': nn.Sequential(
                nn.ConvTranspose2d(nz, nch_g * 4, kernel_size=3, stride=1, padding=0),  # 転置畳み込み
                nn.BatchNorm2d(nch_g * 4),  # バッチノーマライゼーション
                nn.ReLU(),  # 活性化関数 ReLU
            ),  # (B, nz, 1, 1) -> (B, nch_g*4, 3, 3)
            'layer1': nn.Sequential(
                nn.ConvTranspose2d(nch_g * 4, nch_g * 2, kernel_size=3, stride=2, padding=0),
                nn.BatchNorm2d(nch_g * 2),
                nn.ReLU(),
            ),  # (B, nch_g*4, 3, 3) -> (B, nch_g*2, 7, 7)
            'layer2': nn.Sequential(
                nn.ConvTranspose2d(nch_g * 2, nch, kernel_size=3, stride=1, padding=0),
                nn.BatchNorm2d(nch),
                nn.ReLU(),
            ),  # (B, nch_g, 14, 14) -> (B, nch, 28, 28)
        })

    def forward(self, z):
        """
        順方向の演算
        Args:
            z : 入力ベクトル

        Returns:
            torch.Tensor : 生成画像
        """
        for layer in self.layers.values():  # self.layersnの各層で演算を行う
            z = layer(z)
        return z

In [ ]:
# 識別器DのクラスDiscriminator
class Discriminator(nn.Module):
    """
    識別器Dのクラス
    """
    def __init__(self, nch=1, nch_d=128):
        super().__init__()
        self.layers = nn.ModuleDict({
            'layer0': nn.Sequential(
                nn.Conv2d(nch, nch_d, 4, 2, 1),  # 畳み込み
                nn.LeakyReLU(negative_slope=0.2),  # leaky ReLU
            ), # (B, nch, 28, 28) >- (B, nch_d, 14, 14)
            'layer1': nn.Sequential(
                nn.Conv2d(nch_d, nch_d * 2, 4, 2, 1),
                nn.BatchNorm2d(nch_d * 2),
                nn.LeakyReLU(negative_slope=0.2),
            ),  # (B, nch_d, 14, 14) >- (B, nch_d*2, 7, 7)
            'layer2': nn.Sequential(
                nn.Conv2d(nch_d * 2, nch_d * 4, 3, 2, 0),
                nn.BatchNorm2d(nch_d * 4),
                nn.LeakyReLU(negative_slope=0.2),
            ),  # (B, nch_d*2, 7, 7) >- (B, nch_d*4, 3, 3)
            'layer3': nn.Sequential(
                nn.Conv2d(nch_d * 4, 1, 3, 1, 0),
                nn.Sigmoid(), # 活性関数 Sigmoid関数
            )  # (B, nch_d*4, 3, 3) >- (B, 1, 1, 1)
        })

    def forward(self, x):
        for layer in self.layers.values():  # self.layersの各層で演算を行う
            x = layer(x)

        return x.squeeze()